In [37]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [49]:
start_date = '2022-11-21'
end_date = '2022-11-27'

In [50]:
query = f"""
with base as 
(
select
    week(cast(run_date as date))+1 week_number,
    customer_id,
    taxi_lifetime_stage
from
    datasets.iallocator_customer_segments
where
    run_date between date_format(cast('{start_date}' as date)-interval '1' day,'%Y-%m-%d')
    and date_format(cast('{end_date}' as date) - interval '5' day,'%Y-%m-%d') 
    and taxi_lifetime_rides >0
    and day_of_week(cast (run_date as date)) = 7
    --and taxi_lifetime_last_ride_city = 'Kolkata'
    --AND taxi_lifetime_last_ride_city in ('Kolkata','Jaipur','Hyderabad')
),

order_raw as (
    SELECT
        -- s.city_name,
        s.customer_id,
        week(cast(date_parse(yyyymmdd ,'%Y%m%d') as date)) week_number,
        date_trunc('week', date_parse(yyyymmdd ,'%Y%m%d')) as week_start_date,
        service_obj_service_name,
        offer_details_offer_type,
        order_id,
        sub_total,
        discount,
        distance_final_distance
        
    FROM orders.order_logs_snapshot s
    WHERE 
        yyyymmdd >= date_format(cast('{start_date}' as date),'%Y%m%d') AND yyyymmdd <= date_format(cast('{end_date}' as date),'%Y%m%d') 
        AND (spd_fraud_flag != true OR spd_fraud_flag is null)
        and service_obj_service_name in ('Auto','Link')
        and order_status = 'dropped'
        --AND city_name = 'Kolkata'
        --AND s.city_name in ('Kolkata','Jaipur','Hyderabad')

)

select 
    o.week_number,
    o.week_start_date,
    coalesce(base.taxi_lifetime_stage, 'NEW') taxi_lifetime_stage,
    --NET ORDERS
        count(distinct case when service_obj_service_name = 'Link'  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) net_orders_link,
        count(distinct case when service_obj_service_name = 'Auto' and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) net_orders_auto,
        count(distinct case when service_obj_service_name IN ('Link','Auto')  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) net_orders_taxi,
        count(distinct case when service_obj_service_name = 'Auto' and offer_details_offer_type = 'subscription' then order_id end) net_orders_auto_subs,
        count(distinct case when service_obj_service_name = 'Link' and offer_details_offer_type = 'subscription' then order_id end) net_orders_link_subs,
        count(distinct case when service_obj_service_name IN ('Link','Auto') and offer_details_offer_type = 'subscription' then order_id end) taxi_subs_rides,
        
        --DISCOUNTED RIDS 
        --count(distinct case when service_obj_service_name = 'Link'  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) and discount>0 then order_id end) disc_net_orders_link,
        --count(distinct case when service_obj_service_name = 'Auto'  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) and discount>0 then order_id end) disc_net_orders_auto,
        --count(distinct case when service_obj_service_name IN ('Auto','Link')  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) and discount>0 then order_id end) disc_net_orders_taxi,
        
        --RIDES NON-SUBS
        --count(distinct case when service_obj_service_name = 'Link' and distance_final_distance <= 6 and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) link_short_rides,
        --count(distinct case when service_obj_service_name = 'Link' and distance_final_distance >6   and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) link_long_rides,
        --count(distinct case when service_obj_service_name = 'Auto' and distance_final_distance <= 6 and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) auto_short_rides,
        --count(distinct case when service_obj_service_name = 'Auto' and distance_final_distance >6   and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) auto_long_rides,
        --count(distinct case when distance_final_distance >6   and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) taxi_long_rides,
        --count(distinct case when distance_final_distance <=6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then order_id end) taxi_short_rides,

        --RIDES SUBS
        --count(distinct case when service_obj_service_name = 'Link' and distance_final_distance <= 6 and offer_details_offer_type = 'subscription' then order_id end) link_subs_short_rides,
        --count(distinct case when service_obj_service_name = 'Link' and distance_final_distance >6   and offer_details_offer_type = 'subscription' then order_id end) link_subs_long_rides,
        --count(distinct case when service_obj_service_name = 'Auto' and distance_final_distance <= 6 and offer_details_offer_type = 'subscription' then order_id end) auto_subs_short_rides,
        --count(distinct case when service_obj_service_name = 'Auto' and distance_final_distance >6   and offer_details_offer_type = 'subscription' then order_id end) auto_subs_long_rides,
        --count(distinct case when distance_final_distance >6   and offer_details_offer_type = 'subscription' then order_id end) taxi_subs_long_rides,
        --count(distinct case when distance_final_distance <=6  and offer_details_offer_type = 'subscription' then order_id end) taxi_subs_short_rides,

        --DISCOUNT NON-SUBS
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance <= 6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) link_short_discount,
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance >6    and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) link_long_discount,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance <= 6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) auto_short_discount,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance >6    and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) auto_long_discount,
        --sum( case when distance_final_distance >6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) taxi_long_discount,
        --sum( case when distance_final_distance <=6 and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null)  then discount end) taxi_short_discount,
        sum( case when service_obj_service_name = 'Link'  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) discount_link,
        sum( case when service_obj_service_name = 'Auto' and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) discount_auto,
        sum( case when service_obj_service_name IN ('Auto','Link') and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then discount end) discount_taxi,
        
        --DISCOUNT SUBS
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance <= 6  and offer_details_offer_type = 'subscription' then discount end) link_subs_short_discount,
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance >6    and offer_details_offer_type = 'subscription' then discount end) link_subs_long_discount,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance <= 6  and offer_details_offer_type = 'subscription' then discount end) auto_subs_short_discount,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance >6    and offer_details_offer_type = 'subscription' then discount end) auto_subs_long_discount,
        --sum( case when distance_final_distance <= 6  and offer_details_offer_type = 'subscription' then discount end) taxi_subs_short_discount,
        --sum( case when distance_final_distance >6    and offer_details_offer_type = 'subscription' then discount end) taxi_subs_long_discount,
        sum( case when service_obj_service_name = 'Link' and offer_details_offer_type = 'subscription' then discount end) link_subs_discount,
        sum( case when service_obj_service_name = 'Auto' and offer_details_offer_type = 'subscription' then discount end) auto_subs_discount,
        sum( case when offer_details_offer_type = 'subscription' then discount end) taxi_subs_discount,
        
        --SUB-TOTAL NON-SUBS
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance <= 6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) link_short_subtotal,
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance >6    and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) link_long_subtotal,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance <= 6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) auto_short_subtotal,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance >6    and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) auto_long_subtotal,
        --sum( case when distance_final_distance >6   and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) taxi_long_subtotal,
        --sum( case when distance_final_distance <=6  and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) taxi_short_subtotal,
        sum( case when service_obj_service_name = 'Link' and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) link_subtotal,
        sum( case when service_obj_service_name = 'Auto' and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) auto_subtotal,
        sum( case when service_obj_service_name IN ('Auto','Link') and (offer_details_offer_type != 'subscription' or offer_details_offer_type is null) then sub_total end) taxi_subtotal,
        
        --SUB-TOTAL SUBS
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance <= 6  and offer_details_offer_type = 'subscription' then sub_total end) link_subs_short_subtotal,
        --sum( case when service_obj_service_name = 'Link' and distance_final_distance >6    and offer_details_offer_type = 'subscription' then sub_total end) link_subs_long_subtotal,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance <= 6  and offer_details_offer_type = 'subscription' then sub_total end) auto_subs_short_subtotal,
        --sum( case when service_obj_service_name = 'Auto' and distance_final_distance >6    and offer_details_offer_type = 'subscription' then sub_total end) auto_subs_long_subtotal,
        --sum( case when distance_final_distance >6   and offer_details_offer_type = 'subscription' then sub_total end) taxi_subs_long_subtotal,
        --sum( case when distance_final_distance <=6  and offer_details_offer_type = 'subscription' then sub_total end) taxi_subs_short_subtotal,
        sum( case when service_obj_service_name = 'Link' and offer_details_offer_type = 'subscription' then sub_total end) link_subs_subtotal,
        sum( case when service_obj_service_name = 'Auto' and offer_details_offer_type = 'subscription' then sub_total end) auto_subs_subtotal,
        sum( case when service_obj_service_name IN ('Auto','Link') and offer_details_offer_type = 'subscription' then sub_total end) taxi_subs_subtotal
        
from order_raw o
left join base on base.customer_id = o.customer_id and base.week_number = o.week_number
group by 1,2,3
order by 1,3
"""

In [51]:
df = pd.read_sql(query, conn)
df.head(20)

,week_number,week_start_date,taxi_lifetime_stage,net_orders_link,net_orders_auto,net_orders_taxi,net_orders_auto_subs,net_orders_link_subs,taxi_subs_rides,discount_link,...,discount_taxi,link_subs_discount,auto_subs_discount,taxi_subs_discount,link_subtotal,auto_subtotal,taxi_subtotal,link_subs_subtotal,auto_subs_subtotal,taxi_subs_subtotal
0,47,2022-11-21 00:00:00.000,CHURN_OTB,111594,53585,165179,546,3217,3763,329328.0,...,470867.0,36419.0,8683.0,45102.0,6420552.0,6981629.0,13402181.0,188614.0,76094.0,264708.0
1,47,2022-11-21 00:00:00.000,COMMITTED,1107330,603779,1711109,30881,112035,142916,1242348.0,...,1708762.0,1364426.0,452537.0,1816963.0,61671347.0,69823625.0,131494972.0,6133523.0,3527672.0,9661195.0
2,47,2022-11-21 00:00:00.000,DETOX,326464,45002,371466,8932,86810,95742,1699612.0,...,1914071.0,1170778.0,148182.0,1318960.0,14096920.0,4735802.0,18832722.0,4268144.0,866309.0,5134453.0
3,47,2022-11-21 00:00:00.000,DORMANT,460171,244931,705102,638,5205,5843,1495129.0,...,2356694.0,51392.0,10498.0,61890.0,28740661.0,33401464.0,62142125.0,343771.0,93060.0,436831.0
4,47,2022-11-21 00:00:00.000,HANDHOLDING,235910,122955,358865,388,2660,3048,1138847.0,...,1662379.0,34139.0,6957.0,41096.0,14288718.0,16013989.0,30302707.0,175736.0,51287.0,227023.0
5,47,2022-11-21 00:00:00.000,HOOK,516655,229728,746383,3766,19666,23432,1372826.0,...,1890125.0,218467.0,58025.0,276492.0,29914753.0,29561686.0,59476439.0,1143216.0,495385.0,1638601.0
6,47,2022-11-21 00:00:00.000,INACTIVE,218204,79282,297486,49,359,408,946075.0,...,1372673.0,4515.0,848.0,5363.0,14276078.0,11148306.0,25424384.0,21745.0,6911.0,28656.0
7,47,2022-11-21 00:00:00.000,NEW,288418,145760,434178,16,402,418,2991268.0,...,4699571.0,5632.0,299.0,5931.0,19597987.0,19502631.0,39100618.0,26831.0,2699.0,29530.0
8,47,2022-11-21 00:00:00.000,SOFT_CHURN,53675,23925,77600,144,996,1140,204627.0,...,296415.0,10885.0,2187.0,13072.0,3147834.0,3161266.0,6309100.0,61577.0,22118.0,83695.0
9,47,2022-11-21 00:00:00.000,SUSTENANCE,201384,144983,346367,2135,4207,6342,307174.0,...,499012.0,42392.0,32859.0,75251.0,13011161.0,19181701.0,32192862.0,293939.0,297776.0,591715.0


In [48]:
df_46_dump_pan=df


In [53]:
df_47_dump_pan=df
